# Transposed convolutional layers

In fully convolutional networks, such as U-Nets, there are convolutional and transposed convolutional layers. To best understand where the term transposed convolution comes from, it is easiest to first understand that the convolution operation in CNNs (mathematically speaking a cross-correlation) can be writted as a matrix-vector product where the filter is converted into a so-called convolutional matrix and the 2D input is reshaped into a column vector. To perform a transposed convolution we can first write the convolutional matrix by computing the output shape resulting from the transposed convolution. Next, we can use that output shape and treat it as out input to build the convolutional matrix as if we were to perform a convolution. After building the convolutional matrix we can take its transpose and compute the matrix-vector product with the input.

In this notebook I only treat valid and same convolutions and transposed convolutions and only consider filters of shape (nxn) where n is an uneven number.

In [1]:
import numpy as np
import tensorflow as tf # tf version 2.3.0, functions might not work for other versions
from scipy import signal
import random

In [2]:
def filt2matrix(filt, inp, padding="valid"):
    
    '''
    filt: 2d filter to convert to convolutional matrix
    inp: 2d input used to compute the compute the convolution
    padding: must be valid or same
    '''
    
    if padding =="same":
        # compute necessary padding and pad input
        p = int((filt.shape[0]-1)/2)
        inp = np.pad(inp, pad_width=[(p,p),(p,p)])
    
    # Initialize the convolutional matrix
    convMat=np.zeros(((inp.shape[0]-filt.shape[0]+1)*(inp.shape[1]-filt.shape[1]+1),inp.shape[0]*inp.shape[1]))
    
    # Compute the shape of the output after the convolution
    out_shape = (inp.shape[0]-filt.shape[0]+1, inp.shape[1]-filt.shape[1]+1)
    
    # Pad the filter with zeros such that it has the same shape as the input
    filt_pad = np.pad(filt, pad_width=[(0,inp.shape[0]-filt.shape[0]),(0,inp.shape[1]-filt.shape[1])])
    # Flatten the filter to a 1D array
    filt_pad_flat = filt_pad.flatten()
    
    # Initialze counters for row and roll parameter
    rollc, rowc = 0, 0
    
    # Start by filling first row of convMat will the padded and flattened filter
    convMat[0,:]= filt_pad_flat
    # Fill remaining rows 
    for i in range(0,out_shape[0]):
        
        if i>0:
            rollc+=filt.shape[0]-1
            
        for j in range(0,out_shape[1]):
            
            convMat[rowc,:]=np.roll(filt_pad_flat, rollc)
            rowc+=1
            rollc+=1
            
            
    return convMat, inp

def conv2D(filt, inp, padding="valid"):
    '''
    compute convolution
    '''
    
    
    convMat, inpt = filt2matrix(filt, inp, padding)
    
    inpt_col_vec = np.reshape(inpt, (inpt.shape[1]*inpt.shape[0],1))
    
    out = np.dot(convMat, inpt_col_vec)
    out = np.reshape(out, (inpt.shape[0]-filt.shape[0]+1, inpt.shape[1]-filt.shape[1]+1))
    
    return out


def conv2DTranspose(filt, inp, padding='valid'):
    
    '''
    compute transposed convolution
    '''
    
    if padding=='valid':
        o1 = inp.shape[0]+(filt.shape[0]-1)
        o2 = inp.shape[1]+(filt.shape[1]-1)
    elif padding=='same':
        o1, o2 = inp.shape
        
    out_shape = (o1, o2)
    
    if padding=='valid':
        inp2 = np.zeros(out_shape)
        convMat, tmp = filt2matrix(filt, inp2, padding)
        inpt_col_vec = np.reshape(inp, (inp.shape[1]*inp.shape[0],1))
        out = np.dot(convMat.T, inpt_col_vec)
        out = np.reshape(out, (inp.shape[0]+(filt.shape[0]-1), inp.shape[1]+(filt.shape[1]-1)))
    elif padding=='same':
        # compute necessary padding ONLY FOR SAME SIZED FILTERS
        p = int((filt.shape[0]-1)/2)
        convMat, tmp = filt2matrix(filt, inp, padding)
        inpt_col_vec = np.reshape(inp, (inp.shape[1]*inp.shape[0],1))
        out = np.dot(convMat.T, inpt_col_vec)
        out = np.reshape(out, (inp.shape[0]+(filt.shape[0]-1), inp.shape[1]+(filt.shape[1]-1)))
        out = out[p:-p,p:-p]
    
    return out

def tf_conv2DTranspose(filt, inp, padding='valid'):
    
    if padding=='valid':
        o1 = inp.shape[0]+(filt.shape[0]-1)
        o2 = inp.shape[1]+(filt.shape[1]-1)
    elif padding=='same':
        o1, o2 = inp.shape
        
    out_shape = (o1, o2)

    tf_I = tf.convert_to_tensor(np.reshape(inp, (1,inp.shape[0], inp.shape[1],1)), dtype=float)
    tf_K = tf.convert_to_tensor(np.reshape(filt, (filt.shape[0], filt.shape[1],1,1)), dtype=float)
    
    tf_conv = tf.nn.conv2d_transpose(
        tf_I, tf_K, output_shape=[out_shape[0], out_shape[1]], strides=1, padding=padding.upper(),
    )
    tf_conv = tf_conv.numpy()
    tf_conv = np.reshape(tf_conv, (tf_conv.shape[1], tf_conv.shape[2]))
    
    return tf_conv

## Check if convolution works fine for random examples

In [6]:
for i in range(0,1000):
    
    
    
    # chose randomly between valid and same convoltions
    padding = np.random.choice(['valid', 'same'])
    
    i1, i2 = random.randrange(7,21,1), random.randrange(7,21,1)
    inp = np.random.randint(0,101, (i1,i2))
    filt_size = random.randrange(3,7,2)
    filt = np.random.randint(0,9,(filt_size,filt_size))
    
    # Compute transpose conv for both tf and own implementation and check if same
    conv = conv2D(filt,inp,padding=padding)
    scpy_conv = signal.correlate2d(inp,filt, mode=padding)
    
    np.testing.assert_array_equal(conv, scpy_conv)      

## Check if transposed convolution works fine for random examples

In [7]:
for i in range(0,1000):
    # Define random filter size (3 or 5) and fill with random values
    filt_size = random.randrange(3,7,2)
    filt = np.random.randint(0,9,(filt_size,filt_size))
    
    # Define input of random size and random values
    i1, i2 = random.randrange(3,21,1), random.randrange(3,21,1)
    inp = np.random.randint(0,101, (i1,i2))
    
    # chose randomly between valid and same convoltions
    padding = np.random.choice(['valid', 'same'])
    
    # Compute transpose conv for both tf and own implementation and check if same
    conv2DT = conv2DTranspose(filt,inp,padding=padding)
    tf_conv2DT = tf_conv2DTranspose(filt, inp, padding=padding)
    
    np.testing.assert_array_equal(conv2DT, tf_conv2DT)
        